In [1]:
import json
import os
import sys
sys.path.append("../..")

from evals.spelling_by_grade import create_full_spelling_prompt, prepare_grade_spelling_eval
from evals.plot_utils import basic_bar_graph, create_confusion_matrix, create_table, nested_bar_graph
from evals.eval_utils import ModelType
from evals.eval_list import *

words_by_grade = prepare_grade_spelling_eval("../data/GradeSpellingEval.txt", '-')
words_by_grade[3][:10]

ModuleNotFoundError: No module named 'plotly'

In [ ]:
with open("../data/grade_spelling_eval_results_gptj.json") as file:
    eval_results = json.load(file)

In [ ]:
table = create_table(eval_results[FULL_SPELLING_EVAL.name]['2']['data'])
table

In [ ]:
from collections import defaultdict

char_dict = {item: {'correct': 0, 'incorrect': 0} for item in string.ascii_uppercase}

def populate_char_dict(char_dict, answer, response):
    answer, response = answer.split('-'), response.split('-')
    for i, char in enumerate(answer):
        if char in char_dict:
            if len(response) > i and response[i] == answer[i]:
                char_dict[char]['correct'] += 1
            else:
                char_dict[char]['incorrect'] += 1
                break

# Check for the first misspelling in all words without special characters like periods/apostrophes.
# Only the first because otherwise misspellings like R-A-B-B-I-T -> R-A-B-I-T would count as wrong for I and T as well.
for grade in eval_results[FULL_SPELLING_EVAL.name]['2']['data']:
    for word in eval_results[FULL_SPELLING_EVAL.name]['2']['data'][grade]:
        if ''.join(word['answer'].split('-')).isalpha():
            populate_char_dict(char_dict, word['answer'], word['formatted_response'])

print(char_dict)

In [ ]:
import plotly.io as pio

pio.renderers.default = "iframe" # No idea why this works, and the others don't.
char_accuracy_dict = {i: char_dict[i]['correct'] / (char_dict[i]['correct'] + char_dict[i]['incorrect']) for i in char_dict.keys()}
print(char_accuracy_dict)
basic_bar_graph(char_accuracy_dict)

In [ ]:
from collections import defaultdict
import copy

allowed_characters = list(string.ascii_uppercase) + ['_']
char_dict = {item: defaultdict(int) for item in allowed_characters}
char_dict['Other'] = defaultdict(int)

def populate_char_dict(char_dict, answer, response):
    # answer and response are of the form H-E-L-L-O.
    answer, response = answer.split('-'), response.split('-')

    for i, char in enumerate(answer):
        # If response is shorter than answer, treat missing characters as spaces
        if char not in char_dict:
            char = 'Other'
        resp_char = response[i] if len(response) > i else '_'
        if resp_char not in char_dict:
            resp_char = 'Other'
        char_dict[char][resp_char] += 1

        if char != resp_char:
            break  # Stop checking further characters after the first misspelling

# Check for the first misspelling in all words without special characters like periods/apostrophes.
# Only the first because otherwise misspellings like R-A-B-B-I-T -> R-A-B-I-T would count as wrong for I and T as well.
for grade in eval_results[FULL_SPELLING_EVAL.name]['2']['data']:
    for word in eval_results[FULL_SPELLING_EVAL.name]['2']['data'][grade]:
        if ''.join(word['answer'].split('-')).isalpha(): # Ignore non-alphabetic characters like apostrophes.
            populate_char_dict(char_dict, word['answer'], word['formatted_response'])
            
create_confusion_matrix(char_dict)

In [1]:
def get_precision(char_dict, char):
    true_positives = char_dict[char][char]
    false_positives = sum([char_dict[c][char] for c in char_dict if c!= char])
    return 0 if true_positives + false_positives == 0 else round(true_positives / (true_positives + false_positives), 3)

def get_recall(char_dict, char):
    true_positives = char_dict[char][char]
    false_negatives = sum([char_dict[char][c] for c in char_dict if c != char])
    return 0 if true_positives + false_negatives == 0 else round(true_positives / (true_positives + false_negatives), 3)

def get_f1_score(char_dict, char):
    precision, recall = get_precision(char_dict, char), get_recall(char_dict, char)
    return 0 if precision + recall == 0 else round((2 * precision * recall) / (precision + recall), 3)

prf_dict = {char: {'precision': get_precision(char_dict, char), 
                   'recall': get_recall(char_dict, char), 
                   'f1_score': get_f1_score(char_dict, char)} for char in char_dict}
prf_dict
basic_bar_graph({k: prf_dict[k]['precision'] for k in prf_dict.keys()})
basic_bar_graph({k: prf_dict[k]['recall'] for k in prf_dict.keys()})
basic_bar_graph({k: prf_dict[k]['f1_score'] for k in prf_dict.keys()})

NameError: name 'char_dict' is not defined